# Amazon Bedrock AgentCore Runtime で Amazon Bedrock モデルを使用した CrewAI マルチエージェントクルーをホスティング

## 概要

このチュートリアルでは、Amazon Bedrock AgentCore Runtime を使用して既存のマルチエージェントクルーをホスティングする方法を学びます。

CrewAI と Amazon Bedrock モデルの例に焦点を当てます。Amazon Bedrock モデルを使用した Strands Agents については[こちら](../01-strands-with-bedrock-model)、OpenAI モデルを使用した Strands Agents については[こちら](../03-strands-with-openai-model)をご確認ください。


### チュートリアル詳細

| 情報 | 詳細 |
|:--------------------|:-----------------------------------------------------------------------------|
| チュートリアルタイプ | 会話形式 |
| エージェントタイプ | マルチエージェントクルー |
| エージェントフレームワーク | CrewAI |
| LLM モデル | Anthropic Claude Haiku 4.5 |
| チュートリアル構成 | AgentCore Runtime でのエージェントホスティング。CrewAI と Amazon Bedrock モデルの使用 |
| チュートリアル分野 | クロスバーティカル |
| 例の複雑さ | 簡単 |
| 使用 SDK | Amazon BedrockAgentCore Python SDK と boto3 |

### チュートリアルアーキテクチャ

このチュートリアルでは、既存のマルチエージェントクルーを AgentCore Runtime にデプロイする方法を説明します。

デモンストレーション目的で、Amazon Bedrock モデルを使用した CrewAI クルーを使用します。

この例では、リサーチャーとアナリストの2つのエージェントを持つリサーチクルーを使用します。
<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="60%"/>
</div>


### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントホスティング
* Amazon Bedrock モデルの使用
* CrewAI の使用

## 前提条件

このチュートリアルを実行するには、以下が必要です：
* Python 3.10+
* uv パッケージマネージャー
* AWS 認証情報
* Docker が実行中

さらに、いくつかの依存関係をインストールする必要があります：
* Amazon Bedrock AgentCore SDK
* CrewAI
* Langchain community パッケージ
* DuckDuckGo 検索

必要なすべての依存関係を pyproject.toml ファイルにパッケージ化しているので、簡単にインストールできます。

In [ ]:
!uv sync --active --force-reinstall

## マルチエージェントクルーの作成とローカルでの実験

AgentCore Runtime にエージェントをデプロイする前に、実験目的でローカルで開発・実行してみましょう。

このガイドでは、トピックを調査・分析し、包括的なレポートを作成するリサーチクルーの作成を説明します。この実践的な例は、AI エージェントが協力して複雑なタスクを達成する方法を示しています。この例は、CrewAI が直接提供する[入門ガイド](https://docs.crewai.com/en/guides/crews/first-crew)から適応されています。

ローカルアーキテクチャは以下のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>


### エージェント、タスク、クルーの定義

まず、ローカル CrewAI エージェントを定義するアーティファクトを作成します：
* agents.yaml - クルーに関与する2つのエージェントを定義
* tasks.yaml - クルー内のエージェントが実行するタスクを定義
* crew.py - 定義されたタスクに取り組むエージェントで構成されるクルーを定義
* main.py - クルーの実行を開始するローカルエントリーポイント

In [ ]:
import os
os.makedirs('research_crew/config', exist_ok=True)

In [ ]:
%%writefile research_crew/config/agents.yaml
researcher:
  role: >
    Senior Research Specialist for {topic}
  goal: >
    Find comprehensive and accurate information about {topic}
    with a focus on recent developments and key insights
  backstory: >
    あなたは様々なソースから関連情報を見つける才能を持つ経験豊富なリサーチスペシャリストです。
    情報を明確で構造化された方法で整理し、複雑なトピックを他の人にも理解しやすくすることに優れています。
  llm: bedrock/global.anthropic.claude-haiku-4-5-20251001-v1:0

analyst:
  role: >
    Data Analyst and Report Writer for {topic}
  goal: >
    Analyze research findings and create a comprehensive, well-structured
    report that presents insights in a clear and engaging way
  backstory: >
    あなたはデータ解釈と技術文書作成のバックグラウンドを持つ熟練したアナリストです。
    リサーチデータからパターンを特定し、意味のあるインサイトを抽出する才能があり、
    そのインサイトを巧みに作成されたレポートを通じて効果的に伝えることができます。
  llm: bedrock/global.anthropic.claude-haiku-4-5-20251001-v1:0

In [ ]:
%%writefile research_crew/config/tasks.yaml
research_task:
  description: >
    Conduct thorough research on {topic}. Focus on:
    1. Key concepts and definitions
    2. Historical development and recent trends
    3. Major challenges and opportunities
    4. Notable applications or case studies
    5. Future outlook and potential developments

    Make sure to organize your findings in a structured format with clear sections.
  expected_output: >
    A comprehensive research document with well-organized sections covering
    all the requested aspects of {topic}. Include specific facts, figures,
    and examples where relevant.
  agent: researcher

analysis_task:
  description: >
    Analyze the research findings and create a comprehensive report on {topic}.
    Your report should:
    1. State the topic and begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer recommendations or future considerations
    5. Be formatted in a professional, easy-to-read style with clear headings
  expected_output: >
    A polished, professional report on {topic} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.
  agent: analyst
  context:
    - research_task

In [ ]:
%%writefile research_crew/crew.py
from crewai import Agent, Crew, Process, Task
from crewai.project import CrewBase, agent, crew, task
from crewai.agents.agent_builder.base_agent import BaseAgent
from typing import List
from langchain_community.tools import DuckDuckGoSearchRun
from crewai.tools import BaseTool
from crewai_tools import SerperDevTool
from pydantic import Field


class SearchTool(BaseTool):
     name: str = "Search"
     description: str = "Useful for searching the web for information."
     search: DuckDuckGoSearchRun = Field(default_factory=DuckDuckGoSearchRun)

     def _run(self, query: str) -> str:
         """Execute the search query and return results"""
         try:
             return self.search.invoke(query)
         except Exception as e:
             return f"Error performing search: {str(e)}"

@CrewBase
class ResearchCrew():
    """Research crew for comprehensive topic analysis and reporting"""

    agents: List[BaseAgent]
    tasks: List[Task]

    @agent
    def researcher(self) -> Agent:
        return Agent(
            config=self.agents_config['researcher'], # type: ignore[index]
            verbose=True,
            tools=[
                #SerperDevTool()
                SearchTool()
                ]
        )

    @agent
    def analyst(self) -> Agent:
        return Agent(
            config=self.agents_config['analyst'], # type: ignore[index]
            verbose=True
        )

    @task
    def research_task(self) -> Task:
        return Task(
            config=self.tasks_config['research_task'] # type: ignore[index]
        )

    @task
    def analysis_task(self) -> Task:
        return Task(
            config=self.tasks_config['analysis_task'], # type: ignore[index]
            #output_file='output/report.md'
        )

    @crew
    def crew(self) -> Crew:
        """Creates the research crew"""
        return Crew(
            agents=self.agents,
            tasks=self.tasks,
            process=Process.sequential,
            verbose=True,
        )

In [ ]:
%%writefile research_crew/main.py
import os
from research_crew.crew import ResearchCrew

# Create output directory if it doesn't exist
os.makedirs('output', exist_ok=True)

def run():
    """
    Run the research crew.
    """
    inputs = {
        'topic': 'Artificial Intelligence in Healthcare'
    }

    # Create and run the crew
    result = ResearchCrew().crew().kickoff(inputs=inputs)

    # Print the result
    print("\n\n=== 最終レポート ===\n\n")
    print(result.raw)


if __name__ == "__main__":
    run()

### ローカルでのクルー呼び出し

最後に、CrewAI CLI を使用してローカルでクルーを起動できます。または、ローカルエントリーポイントの main.py を直接実行することもできます。これには数分かかる場合があります。

In [ ]:
!crewai run

## マルチエージェントクルーを Amazon Bedrock AgentCore にデプロイ

本番グレードのエージェントアプリケーションでは、クルーをクラウドで実行する必要があります。そのため、クルーを Amazon Bedrock AgentCore にデプロイします。

ここでのアーキテクチャは以下のようになります：

<div style="text-align:left">
     <img src="images/architecture_local.png" width="60%"/>
</div>

クルーを AgentCore にデプロイするには、以下の手順を実行します：

### リモートエントリーポイント

まず、リモートエントリーポイントを作成します。AgentCore Runtime では、エージェントの呼び出し部分を @app.entrypoint デコレーターで装飾し、ランタイムのエントリーポイントとして使用します。これには以下も含まれます：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* `app = BedrockAgentCoreApp()` でコード内で App を初期化
* 呼び出し関数を `@app.entrypoint` デコレーターで装飾
* `app.run()` で AgentCore Runtime にエージェントの実行を制御させる

### 裏側で何が起きているか？

`BedrockAgentCoreApp` を使用すると、自動的に以下が行われます：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するために必要な `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントにとって非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラーハンドリングを管理

In [ ]:
%%writefile research_crew/research_crew.py
import os
from research_crew.crew import ResearchCrew

# ---------- Agentcore インポート --------------------
from bedrock_agentcore.runtime import BedrockAgentCoreApp

app = BedrockAgentCoreApp()
#------------------------------------------------


@app.entrypoint
def agent_invocation(payload, context):
    """エージェント呼び出しのハンドラー"""
    print(f'ペイロード: {payload}')
    try: 
        # デフォルト付きでペイロードからユーザーメッセージを抽出
        user_message = payload.get("prompt", "Artificial Intelligence in Healthcare")
        print(f"処理中のトピック: {user_message}")
        
        # クルーインスタンスを作成し、同期的に実行
        research_crew_instance = ResearchCrew()
        crew = research_crew_instance.crew()
        
        # 非同期ではなく同期 kickoff を使用 - これによりすべてのイベントループの問題を回避
        result = crew.kickoff(inputs={'topic': user_message})

        print("コンテキスト:\n-------\n", context)
        print("結果（Raw）:\n*******\n", result.raw)
        
        # json_dict が存在する場合は安全にアクセス
        if hasattr(result, 'json_dict'):
            print("結果（JSON）:\n*******\n", result.json_dict)
        
        return {"result": result.raw}
        
    except Exception as e:
        print(f'例外が発生しました: {e}')
        return {"error": f"An error occurred: {str(e)}"}

if __name__ == "__main__":
    app.run()

### エージェントを AgentCore Runtime にデプロイ

`CreateAgentRuntime` 操作は、コンテナイメージ、環境変数、暗号化設定を指定できる包括的な設定オプションをサポートしています。また、クライアントがエージェントと通信する方法を制御するために、プロトコル設定（HTTP、MCP）と認可メカニズムも設定できます。

**注意：** 運用のベストプラクティスは、コードをコンテナとしてパッケージ化し、CI/CD パイプラインと IaC を使用して ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCore Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore Runtime にデプロイします。

#### AgentCore Runtime デプロイの設定

まず、スターターツールキットを使用して、エントリーポイント、先ほど作成した実行ロール、および requirements ファイルで AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

AgentCore configure は、ワークロードが実行される Docker コンテナのブループリントを保持する Dockerfile と、エージェントワークロードの設定を保持する .bedrock_agentcore.yaml を生成するために必要です。設定ステップ中に、アプリケーションコードに基づいて Docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "research_crew_getting_started"
response = agentcore_runtime.configure(
    entrypoint="research_crew/research_crew.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    region=region,
    agent_name=agent_name
)
response

#### エージェントを AgentCore Runtime に起動：リモートエージェントワークロードのデプロイ

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。AgentCore launch はエージェントワークロードをクラウドにデプロイします。これには、Docker イメージの作成と ECR へのプッシュ、およびエンドポイントの準備が含まれます。

<div style="text-align:left">
    <img src="images/launch.png" width="85%"/>
</div>

In [ ]:
launch_result = agentcore_runtime.launch()

#### AgentCore Runtime のステータス確認

AgentCore Runtime をデプロイしたので、デプロイステータスを確認しましょう。

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### boto3 を使用した AgentCore Runtime の呼び出し

AgentCore Runtime が作成されたら、任意の AWS SDK で呼び出すことができます。例えば、boto3 の `invoke_agent_runtime` メソッドを使用できます。これは長時間実行されるエージェントなので、デフォルトの `retries`、`connect_timeout`、`read_timeout` を上書きしています。

<div style="text-align:left">
    <img src="images/invoke.png" width=85%"/>
</div>

In [ ]:
from botocore.config import Config

# リトライとタイムアウトを設定
config = Config(
    retries={
        'max_attempts': 10,  # 最大リトライ回数を10に増加（デフォルトは4）
        'mode': 'adaptive'   # オプション: 'legacy', 'standard', 'adaptive'
    },
    connect_timeout=600,      # 接続タイムアウト（秒）（デフォルトは60）
    read_timeout=3000         # 読み取りタイムアウト（秒）（デフォルトは60）
)

In [ ]:
import boto3
import json
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region,
    config=config
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "Artificial Intelligence in Healthcare"})
)

response_body = boto3_response['response'].read()
response_data = json.loads(response_body)
response_data

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう。

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

## おめでとうございます！